# code for calculating researcher needs

In [36]:
import pandas
import pygsheets
import numpy

In [37]:
credentials_directory = '/Users/baird/Dropbox/_google-api/'
gc = pygsheets.authorize(client_secret=credentials_directory+'client_secret.json')
spreadsheet = gc.open_by_key('1foPLE6K-uqFlaYgLPAUxzeXfDO5wOOqE7tibNHeqTek')

#spreadsheet[1] "Gas Pipelines" tab is the second index
gas_pipes = spreadsheet.worksheet('title','Gas pipelines').get_as_df(start='A2')
oil_pipes = spreadsheet.worksheet('title', 'Oil/NGL pipelines').get_as_df(start='A2')

pipes_df_orig = pandas.concat([oil_pipes, gas_pipes], ignore_index=True)

#get country ratios sheet
country_ratios_df = spreadsheet.worksheet('title', 'Country ratios by pipeline').get_as_df()

In [38]:
# replace -- entries with NaN
pipes_df_orig = pipes_df_orig.replace('--', numpy.nan)
pipes_df_orig = pipes_df_orig[pipes_df_orig['PipelineName']!='']
#pipes_df_orig = pipes_df_orig[pipes_df_orig['Wiki']!='']
#country_ratios_df.replace('--', numpy.nan, inplace=True)

In [39]:
status_list = ['Proposed', 'Construction', 'Shelved', 'Cancelled', 
               'Operating', 'Idle', 'Mothballed', 'Retired']
country_list = sorted(list(set(country_ratios_df['Country'])))
region_list = sorted(list(set(country_ratios_df['Region'])))

## how many oil pipelines per region?

In [40]:
pipes_df_orig.loc[pipes_df_orig.Fuel.isin(['Oil','NGL'])].shape

(979, 88)

In [57]:
projectid_count_df = pandas.DataFrame(pipes_df_orig.loc[pipes_df_orig.Fuel.isin(['Oil','NGL'])].groupby(['Status','StartRegion'])['ProjectID'].count())

In [54]:
projectid_count_df.unstack().transpose().replace(numpy.nan,0)

Status                                     Cancelled  Construction  Idle  \
          StartRegion                                                      
ProjectID Australia and New Zealand              0.0           0.0   0.0   
          East Asia                              4.0           8.0   0.0   
          Eurasia                                7.0           5.0   0.0   
          Europe                                11.0           1.0   2.0   
          Latin America and the Caribbean        2.0           0.0   0.0   
          Middle East and North Africa           8.0          10.0   0.0   
          North America                         43.0           8.0   0.0   
          SE Asia                                0.0           1.0   0.0   
          South Asia                             2.0           4.0   0.0   
          Sub-Saharan Africa                     1.0           1.0   0.0   

Status                                     Mothballed  Operating  Proposed  \
          StartRegion                                                        
ProjectID Australia and New Zealand               0.0       12.0       0.0   
          East Asia                               0.0      108.0      11.0   
          Eurasia                                 1.0       76.0       5.0   
          Europe                                  0.0       34.0       3.0   
          Latin America and the Caribbean         1.0       53.0       3.0   
          Middle East and North Africa            2.0      126.0       2.0   
          North America                           0.0      287.0      27.0   
          SE Asia                                 0.0       15.0       2.0   
          South Asia                              0.0        9.0       1.0   
          Sub-Saharan Africa                      0.0       35.0       3.0   

Status                                     Retired  Shelved  
          StartRegion                                        
ProjectID Australia and New Zealand            1.0      0.0  
          East Asia                           20.0      0.0  
          Eurasia                              1.0      0.0  
          Europe                               0.0      0.0  
          Latin America and the Caribbean      0.0      2.0  
          Middle East and North Africa         1.0      1.0  
          North America                        2.0     10.0  
          SE Asia                              0.0      1.0  
          South Asia                           0.0      1.0  
          Sub-Saharan Africa                   0.0      1.0

## country-specific calculations

### for Nagwa or Arabic language expert

In [61]:
arabic_speaking_countries = ['Algeria', 
                             'Bahrain', 
                           'Chad', 
                           'Comoros', 
                           'Djibouti', 
                           'Egypt', 
                           'Iraq', 
                           'Jordan', 
                           'Kuwait', 
                           'Lebanon', 
                           'Libya', 
                           'Mauritania', 
                           'Morocco', 
                           'Oman', 
                           'Qatar', 
                           'Saudi Arabia', 
                           'Somalia', 
                           'Sudan', 
                           'Syria', 
                           'Tunisia', 
                           'United Arab Emirates', 
                           'Yemen']

In [62]:
pipes_df_orig.loc[(pipes_df_orig.Fuel.isin(['Oil','NGL']))&
                  (pipes_df_orig.StartCountry.isin(arabic_speaking_countries))].shape

(141, 88)

In [72]:
pipes_df_orig.loc[(pipes_df_orig.Fuel.isin(['Oil','NGL']))&
                  (pipes_df_orig.StartCountry.isin(arabic_speaking_countries))&
                  (pipes_df_orig.Status.isin(['Proposed','Construction']))].shape

(7, 88)

### Russian speaking countries

In [63]:
russian_speaking_countries = ['Russia', 
                              'Belarus', 
                              'Kyrgyzstan',
                              'Kazakhstan', 
                              'Ukraine', 
                              'Azerbaijan', 
                              'Estonia', 
                              'Georgia', 
                              'Latvia', 
                              'Lithuania', 
                              'Moldova', 
                              'Tajikistan', 
                              'Turkmenistan', 
                              'Uzbekistan']

In [64]:
pipes_df_orig.loc[(pipes_df_orig.Fuel.isin(['Oil','NGL']))&
                  (pipes_df_orig.StartCountry.isin(russian_speaking_countries))].shape

(100, 88)

In [70]:
pipes_df_orig.loc[(pipes_df_orig.Fuel.isin(['Oil','NGL']))&
                  (pipes_df_orig.StartCountry.isin(russian_speaking_countries))&
                  (pipes_df_orig.Status.isin(['Proposed','Construction']))].shape

(12, 88)

In [68]:
asia_gas_tracker_countries = [ \
                                'Bangladesh',
                                'Brunei',
                                'Cambodia',
                                #'China',
                                #'Hong Kong',
                                'India',
                                'Indonesia',
                                'Japan',
                                #'Macao',
                                'Malaysia',
                                'Myanmar',
                                'Pakistan',
                                'Philippines',
                                'Singapore',
                                'South Korea',
                                'Sri Lanka',
                                'Taiwan',
                                'Thailand',
                                'Vietnam'
                             ]

In [69]:
pipes_df_orig.loc[(pipes_df_orig.Fuel.isin(['Oil','NGL']))&
                  (pipes_df_orig.StartCountry.isin(asia_gas_tracker_countries))].shape

(38, 88)

In [73]:
pipes_df_orig.loc[(pipes_df_orig.Fuel.isin(['Oil','NGL']))&
                  (pipes_df_orig.StartCountry.isin(asia_gas_tracker_countries))&
                  (pipes_df_orig.Status.isin(['Proposed','Construction']))].shape

(8, 88)

### China/Taiwan/Hong Kong/Macao

In [74]:
pipes_df_orig.loc[(pipes_df_orig.Fuel.isin(['Oil','NGL']))&
                  (pipes_df_orig.StartCountry.isin(['China','Taiwan','Hong Kong','Macao']))].shape

(148, 88)

In [75]:
pipes_df_orig.loc[(pipes_df_orig.Fuel.isin(['Oil','NGL']))&
                  (pipes_df_orig.StartCountry.isin(['China','Taiwan','Hong Kong','Macao']))&
                  (pipes_df_orig.Status.isin(['Proposed','Construction']))].shape

(19, 88)